# folder data 處理

In [1]:
import pandas as pd
import os
root = 'D:\\WorkSpace\\JupyterWorkSpace\\DataSet\\Image-Classification\\AOI自動光學瑕疵檢測\\Stage1\\'
# train_df = pd.read_csv(os.path.join(root, 'train.csv'))
test_df = pd.read_csv(os.path.join(root, 'test.csv'))

# def img_path(id):
#     return os.path.join(root, '%s_images\\%s' % ('train', id))
# def cls_path(id, cls):
#     cls_path = os.path.join(root, '%s_images' % 'train', '%s\\%s'%(cls,id) )
#     image_path = os.path.join(root, '%s_images' % 'train', '%s'%(id) )
#     return cls_path
      
# train_df['image_path'] = train_df['ID'].apply(img_path)
# train_df['cls_path'] = train_df.apply(lambda x : cls_path(x['ID'],x['Label']),axis=1)

# train_df.iloc[0]['image_path']
# train_df.iloc[0]['cls_path']

# def test_img_path(id):
#     return os.path.join(root, '%s_images\\%s' % ('test', id))
# def cls_path(id):
#     cls_path = os.path.join(root, '%s_images' % 'test', '%s\\%s'%(0,id) )
#     image_path = os.path.join(root, '%s_images' % 'test', '%s'%(id) )
#     return cls_path
      
# test_df['image_path'] = test_df['ID'].apply(test_img_path)
# test_df['cls_path'] = test_df.apply(lambda x : cls_path(x['ID']),axis=1)
# test_df.iloc[0]['image_path']
# test_df.iloc[0]['cls_path']

In [2]:
import shutil
from tqdm import tqdm

# with tqdm(total=len(train_df)) as pbar:
#     for i in range(len(train_df)):
#         series = train_df.iloc[i]
#         source, dest = series['image_path'], series['cls_path']
#         shutil.move(source,dest)
#         pbar.update(1)       
        
# with tqdm(total=len(test_df)) as pbar:
#     for i in range(len(test_df)):
#         series = test_df.iloc[i]
#         source, dest = series['image_path'], series['cls_path']
#         shutil.move(source,dest)
#         pbar.update(1)           

# pytorch ImageFolder Dataset 

In [3]:
import time
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.optim as optim

#加上transforms
transform=transforms.Compose([
    transforms.Resize((512,512), interpolation=3), #缩放图片，保持长宽比不变，最短边的长为224像素,
#     transforms.CenterCrop(512), #从中间切出 224*224的图片    
#     transforms.RandomResizedCrop(512),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #将图片转换为Tensor,归一化至[0,1]
    transforms.Normalize([0.5], [0.5])
])

full_dataset = ImageFolder(os.path.join(root, 'train_images'),transform=transform)
pred_modelset = ImageFolder(os.path.join(root, 'test_images'),transform=transform)
# test_dataset = ImageFolder('D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Dev',transform=transform)

# size = len(full_dataset)  # total number of examples
# test_size = int(0.1 * size)  # take ~10% for test
# test_set = torch.utils.data.Subset(full_dataset, range(test_size))  # take first 10%
# train_set = torch.utils.data.Subset(full_dataset, range(test_size, size))  # take the rest   

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.dataset.random_split(full_dataset, [train_size, test_size])

In [4]:
train_dataset.dataset.classes

['0', '1', '2', '3', '4', '5']

In [5]:
#输出第0张图片的大小
print(train_dataset[0][0].size())

torch.Size([3, 512, 512])


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# print(torch.cuda.get_device_name(device))

cuda:0


# 創建數據加載器

In [7]:
from torch.utils.data import DataLoader

torch.manual_seed(1)
BATCH_SIZE = 4

train_loader = DataLoader(
    dataset=train_dataset,# TensorDataset类型数据集
    batch_size=BATCH_SIZE,# mini batch size
    shuffle=True,# 设置随机洗牌
    num_workers=5# 加载数据的进程个数
)

test_loader = DataLoader(
    dataset=test_dataset,# TensorDataset类型数据集
    batch_size=BATCH_SIZE,# mini batch size
    shuffle=True,# 设置随机洗牌
    num_workers=5# 加载数据的进程个数
)

pred_loader = DataLoader(
    dataset=pred_modelset,# TensorDataset类型数据集
    batch_size=1,# mini batch size
    num_workers=5# 加载数据的进程个数
)
num_classes = len(train_dataset.dataset.classes)

In [8]:
batch = next(iter(train_loader))

In [9]:
print(batch[0].size())
print(batch[1].size())

torch.Size([4, 3, 512, 512])
torch.Size([4])


# 構建網絡結構

In [10]:
# 两层卷积
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 使用序列工具快速构建
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(128 * 128 * 32, 2)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0), -1)  # reshape
        out = self.fc(out)
        return out

In [11]:
# 3x3 卷积
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)


# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(128 * 128, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
# 加載預訓練模型
model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features

# 把resnet的全连接层fc 替换成自己设置的线性层nn.Linear
# 输入维度是resnet.fc.in_features, 输出是196维
model_ft.fc = nn.Linear(num_ftrs, num_classes)

In [13]:
from torchvision import models
# cnn = CNN()
# cnn = ResNet(ResidualBlock, [2, 2, 2, 2])
cnn = model_ft
print(cnn)
if torch.cuda.is_available():
    cnn = cnn.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

# 選擇損失函數

In [14]:
num_epochs = 10
LEARNING_RATE = 0.01

# 选择损失函数和优化方法
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=LEARNING_RATE, momentum=0.9)

# 将数据处理成Variable, 如果有GPU, 可以转成cuda形式
def get_variable(x):
    x = Variable(x)
    return x.cuda() if torch.cuda.is_available() else x

# 評估模型

In [15]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    eval_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images = get_variable(images)
            labels = get_variable(labels)

            outputs = model(images)
            eval_loss += loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    eval_loss = eval_loss / total
#     print('Accuracy of the network on the test images: %d %%' % (
#         test_acc))
    return test_acc, eval_loss

# 訓練

In [16]:
losses = []
accuracies = []
test_accuracies = []
# set the model to train mode initially
cnn.train()

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)


for epoch in range(num_epochs):
    since = time.time()
    running_loss = 0.0
    running_correct = 0.0
    for step, (images, labels) in enumerate(train_loader):
        images = get_variable(images)
        labels = get_variable(labels)

        cnn.train()
        outputs = cnn(images)
        loss = loss_func(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # calculate the loss/acc later
        running_loss += loss.item()
        running_correct += (labels==predicted).sum().item()

    epoch_duration = time.time()-since
    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100/BATCH_SIZE*running_correct/len(train_loader)
    print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
    
    losses.append(epoch_loss)
    accuracies.append(epoch_acc)

    # switch the model to eval mode to evaluate on test data
    cnn.eval()
    test_acc, test_loss = eval_model(cnn)
    print("eval_loss: %.4f, eval_acc: %.4f" % (test_loss, test_acc))
    test_accuracies.append(test_acc)
    
     # re-set the model to train mode after validating
    cnn.train()
    scheduler.step(test_acc)
    since = time.time()
    
print('Finished Training')


Epoch 1, duration: 184 s, loss: 1.4574, acc: 48.2690
eval_loss: 0.8274, eval_acc: 64.3281
Epoch 2, duration: 179 s, loss: 0.8489, acc: 69.2631
eval_loss: 7.9796, eval_acc: 64.4269
Epoch 3, duration: 177 s, loss: 0.5831, acc: 79.4510
eval_loss: 4.1125, eval_acc: 58.3004
Epoch 4, duration: 173 s, loss: 0.4941, acc: 82.2700
eval_loss: 1.0336, eval_acc: 77.6680
Epoch 5, duration: 173 s, loss: 0.3670, acc: 87.0425
eval_loss: 1.2433, eval_acc: 77.7668
Epoch 6, duration: 174 s, loss: 0.2014, acc: 92.8042
eval_loss: 1.6279, eval_acc: 78.2609
Epoch 7, duration: 186 s, loss: 0.1586, acc: 94.7082
eval_loss: 2.8207, eval_acc: 74.1107
Epoch 8, duration: 182 s, loss: 0.1470, acc: 94.4857
eval_loss: 1.0765, eval_acc: 78.0632
Epoch 9, duration: 180 s, loss: 0.1220, acc: 95.5737
eval_loss: 0.9694, eval_acc: 82.2134
Epoch 10, duration: 176 s, loss: 0.1109, acc: 96.3403
eval_loss: 1.1879, eval_acc: 77.8656
Finished Training


In [17]:
# Resize((512,512)
# Epoch 10, duration: 1372 s, loss: 0.4933, acc: 77.9643
# eval_loss: 0.1290, eval_acc: 76.2500

# 保存和加載整個網絡

In [18]:
# 保存和加载整个模型, 包括: 网络结构, 模型参数等
torch.save(cnn, 'model.pkl')
cnn = torch.load('model.pkl')

# 保存和加載網絡中的參數
torch.save(cnn.state_dict(), 'params.pkl')
cnn.load_state_dict(torch.load('params.pkl'))



IncompatibleKeys(missing_keys=[], unexpected_keys=[])

# 預測

In [19]:
total_pred = []
with tqdm(total=len(pred_loader)) as pbar:
    for idx, (images, _) in enumerate(pred_loader):       
        images = get_variable(images)
        
        cnn.eval()
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total_pred.append(predicted.item())
        pbar.update(1)

100%|██████████| 19278/19278 [05:24<00:00, 59.44it/s]


In [20]:
test_df['Label'] = total_pred

In [21]:
test_df

,ID,Label
0,test_00001.jpg,1
1,test_00002.jpg,3
2,test_00003.jpg,5
3,test_00004.jpg,5
4,test_00005.jpg,0
5,test_00006.jpg,0
6,test_00007.jpg,0
7,test_00008.jpg,0
8,test_00009.jpg,0
9,test_00010.jpg,0


In [24]:
test_df.to_csv('pred.csv', index=False)